## Imports 🙅🏻‍♂️🙅🏻‍♀️

In [ ]:
import matplotlib
!pip install opencv-python
import matplotlib.pyplot as ajeeb
import cv2
import math
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import image as mpimg
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np
%matplotlib inline
from scipy import linalg
from scipy.linalg import null_space
# !pip install imageio
import imageio


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## TASK-02 (45 MARKS)🕷️

### Pre-Processing (2 Marks)
- Justify each step of your pre-processing in comments
- Save the final pre-process images to the same directory

In [ ]:
# Add your code here
# Load the two images
def normalize_image(image):
    # I need to convert the resized images to grayscaling
    # We need to do homography matrix calculations ahead and as in the tutorial grayscaling the images is necessary
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    normalize_image = cv2.normalize(gray_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    # I need to resize the images to the same dimensions so that they both come to a common scale and same hieght and width
    # I did this to make it compulsory that the images have the same scale
    # Resizing the image will also help in lower values of the matrices when doing homography matrix calculations
    normalize_image= cv2.resize(image, (600, 600))
    return normalize_image

image1 = cv2.imread(r"C:\Users\dell\Documents\25100287_PA3\topview.jpeg")
image2 = cv2.imread(r"C:\Users\dell\Documents\25100287_PA3\warped_370.jpg")

image1 = normalize_image(image1)
image2 = normalize_image(image2)




### Reading image points

Load the first image you're about to stitch and mark some points that you find in the second image too. The `point_reader()` function from `StudentID_PA3.py` will be helpful.

### Reading the first image's points (4 Marks)

In [ ]:
def point_reader(img):
    '''
    img - The image to be marked points on
    '''
    img_with_points = img.copy()

    points = []
    def click_event(event, x, y, flags, param):
        nonlocal points
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(img_with_points, (x, y), 5, (0, 255, 0), -1)
            points.append((x, y))

            cv2.imshow('Image', img_with_points)

    cv2.imshow('Image', img_with_points)
    cv2.setMouseCallback('Image', click_event)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    points_array = np.array(points)

    return points_array

In [ ]:
# Save your points in an np.array
img1_points = point_reader(image1)
print(img1_points)

[[325 355]
 [345 296]
 [327 272]
 [343 331]]


### Reading the first image's points (4 Marks)

In [ ]:
# Save your points in an np.array
img2_points = point_reader(image2)
print(img2_points)

[[435 406]
 [439 332]
 [439 286]
 [438 253]]


### Finding Homography (25 Marks)
You are required to calculate the homography matrix H.
$$
\begin{bmatrix}
x'  \\
y'  \\
h
\end{bmatrix}
=
\begin{bmatrix}
h_1 & h_2 & h_3 \\
h_4 & h_5 & h_6 \\
h_7 & h_8 & h_9
\end{bmatrix}
\times
\begin{bmatrix}
x  \\
y  \\
1
\end{bmatrix}
$$
You can use the strategy from WA-1, Q#7 to solve this =)

In [ ]:
# We are creating the canvas which is essentially the two images joined together
row1, col1 = image1.shape[:2]
row2, col2 = image2.shape[:2]

canvas = np.zeros((max(row1, row2), col1 + col2, 3), dtype='uint8')

# We are placing the first image on the left side of the canvas
canvas[:row1, :col1, :] = image1
# We are placing the second image on the right side of the canvas
canvas[:row2, col1:col1 + col2, :] = image2


In [ ]:
img2_points[:,0] = img2_points[:,0] + col1

In [ ]:
# DUMP YOUR CODE
# We will now zip the points together
points = list(zip(img1_points, img2_points))

homoArray = []
for point1, point2 in points:
    homoArray.append([point1[0], point1[1], 1, 0, 0, 0, -point2[0] * point1[0], -point2[0] * point1[1], -point2[0]])
    homoArray.append([0, 0, 0, point1[0], point1[1], 1, -point2[1] * point1[0], -point2[1] * point1[1], -point2[1]])

homoArray = np.array(homoArray)
print(homoArray)

[[    325     355       1       0       0       0 -336375 -367425   -1035]
 [      0       0       0     325     355       1 -131950 -144130    -406]
 [    345     296       1       0       0       0 -358455 -307544   -1039]
 [      0       0       0     345     296       1 -114540  -98272    -332]
 [    327     272       1       0       0       0 -339753 -282608   -1039]
 [      0       0       0     327     272       1  -93522  -77792    -286]
 [    343     331       1       0       0       0 -356034 -343578   -1038]
 [      0       0       0     343     331       1  -86779  -83743    -253]]


### Calculating SVD
This part has been done for you 🙌🏻

In [ ]:
# List of useful functions
u, s, vh = np.linalg.svd(homoArray) # Replace it with your H matrix
vh = np.transpose(vh)
P = vh[:,len(vh[0])-1]

P = np.array([P[0:3], P[3:6], P[6:9]])
print("Homography matrix\n",P)

Homography matrix
 [[-1.80520470e-03 -1.09855567e-03  9.68655843e-01]
 [-4.32998226e-04 -3.12163571e-04  2.48395326e-01]
 [-1.73697989e-06 -1.05766854e-06  9.32239157e-04]]


In [ ]:
import cv2
import numpy as np

# Correspondences for the top view image
points_top_view = np.array([[325, 355], [345, 296], [327, 272], [343, 331]], dtype=np.float32)

# Correspondences for the original image
points_original = np.array([[435, 406], [439, 332], [439, 286], [438, 253]], dtype=np.float32)

# Homography matrix
homography_matrix = np.array([[-1.80520470e-03, -1.09855567e-03, 9.68655843e-01],
                              [-4.32998226e-04, -3.12163571e-04, 2.48395326e-01],
                              [-1.73697989e-06, -1.05766854e-06, 9.32239157e-04]])

# GPS coordinates for the corresponding points in both images
gps_top_view = np.array([31.470009, 74.405807], dtype=np.float32)
gps_original = np.array([31.469930, 74.405771], dtype=np.float32)

# Use the homography matrix to map original image points to the top view
points_original_reshape = points_original.reshape(-1, 1, 2)
mapped_points_original = cv2.perspectiveTransform(points_original_reshape, homography_matrix)

# Calculate deviation/error using Euclidean distance
pixel_error = np.mean(np.linalg.norm(mapped_points_original.squeeze() - points_top_view, axis=1))

# Use the homography matrix to transform original GPS coordinates to top view GPS coordinates
gps_original_reshape = gps_original.reshape(-1, 1, 2)
mapped_gps_original = cv2.perspectiveTransform(gps_original_reshape, homography_matrix)

# Calculate deviation/error in GPS coordinates
gps_error = np.linalg.norm(mapped_gps_original.squeeze() - gps_top_view)

# Calculate error percentage
pixel_error_percentage = (pixel_error / np.mean(np.linalg.norm(points_top_view, axis=1))) * 100

print(f"Pixel Deviation/Error: {pixel_error} pixels")
print(f"GPS Deviation/Error: {gps_error} degrees")
print(f"pixel_error_percentage: {pixel_error_percentage}")

Pixel Deviation/Error: 408.98034 pixels
GPS Deviation/Error: 725.4488 degrees
pixel_error_percentage: 31.30375
